#### 문제 3-1 : 콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [1]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

18


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings

print("==> 0. 문서 로딩 → PDF 읽기...")
file_path = "../data/콘텐츠분쟁해결_사례.pdf"
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==> 0. 문서 로딩 → PDF 읽기...
  총 109페이지 로드 완료


In [3]:
print("==> 1. 문서 분할 설정")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 법률 사례는 1500자로 설정
    chunk_overlap=300,      # 사례 맥락 보존을 위해 300자로 설정
    separators=[
        "\n [사건개요]",  # 법률 문서 섹션 구분자
        "\n [쟁점사항]",  # 쟁점 부분 구분
        "\n [처리경위]",  # 처리 과정 구분
        "\n [처리결과]",  # 결과 부분 구분
        "\n\n", "\n", ".", " ",""
    ] 
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")
print(type(chunks[0]))

==> 1. 문서 분할 설정
  104개 청크 생성 완료
  평균 청크 길이: 753자
<class 'langchain_core.documents.base.Document'>


In [4]:
print("==> 2. 벡터화 → 임베딩 생성")
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

print("==> 3. 벡터스토어 구축 → FAISS에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f"  FAISS 벡터스토어 생성 완료 (총 {len(chunks)}개 청크 벡터화)")

# 문제 3-1 전용 폴더명으로 저장
save_path = "faiss_db_case3_1"
vectorstore.save_local(save_path)
print(f"  로컬 경로 '{save_path}'에 저장 완료")

print("==> 4. 검색기 설정 → 유사 문서 검색 준비")
retriever = vectorstore.as_retriever(
    search_type="similarity",     # 문제 가이드 기준
    search_kwargs={"k": 5}        # 상위 5개 관련 문서 검색
)
print("  Retriever 초기화 완료")

==> 2. 벡터화 → 임베딩 생성
==> 3. 벡터스토어 구축 → FAISS에 저장
  FAISS 벡터스토어 생성 완료 (총 104개 청크 벡터화)
  로컬 경로 'faiss_db_case3_1'에 저장 완료
==> 4. 검색기 설정 → 유사 문서 검색 준비
  Retriever 초기화 완료


In [10]:
print("==> 5. LLM 설정 → 답변 생성기 초기화")
llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.2,
    max_tokens=1200 
)
print("  LLM 초기화 완료:", llm)

==> 5. LLM 설정 → 답변 생성기 초기화
  LLM 초기화 완료: client=<openai.resources.chat.completions.completions.Completions object at 0x000002B0B763CC80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B0B763C380> model_name='solar-pro' temperature=0.2 model_kwargs={} max_tokens=1200 upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


In [11]:
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               # 실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  # 한계 인정

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("  프롬프트 설정 완료")

  프롬프트 설정 완료


In [12]:
# ===================================
# 6. QA 체인 생성
# ===================================
print("\n==> 6. QA 체인 생성 → RAG 파이프라인 구축")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  QA 체인 생성 완료!")

# ===================================
# 7. 테스트 질문들
# ===================================
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 60)
print(" 콘텐츠분쟁해결 RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 8. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/{len(test_questions)}】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")

    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]

    print(f"\n 답변:")
    print("-" * 50)
    print(answer)

    # 참조 문서 정보 (앞에서 3개만 표시)
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")

    print("\n" + "-" * 40)


==> 6. QA 체인 생성 → RAG 파이프라인 구축
  QA 체인 생성 완료!

 콘텐츠분쟁해결 RAG 시스템 테스트

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사례 해결 방안  

#### 1. **사례 기반 분석**  
제시된 사례들을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 결과가 달라집니다:  
- **계정 소유권 확인**: 계정 명의자와 실소유자가 다른 경우, 게임사는 복구 요청을 거절할 수 있습니다(2006년 사례).  
- **오류 입증 가능성**: 시스템 오류가 객관적으로 확인되지 않거나, 다른 사용자에게 동일한 문제가 발생하지 않은 경우 복구가 어렵습니다(2009년 사례).  
- **게임사의 약관 및 대응 태도**: 프로그램 오류를 인정하고 복구한 사례(2006년 프로그램 오류 사례)도 있으나, 이는 게임사의 재량에 따릅니다.  
- **아이템의 법적 성격**: 게임 아이템은 일반적으로 "전자적 데이터"로 간주되며, 민법상 재산권 적용이 제한적입니다(2007년 해킹 아이템 사례 참조).  

---

#### 2. **법적 근거**  
- **민법 제250조(도품·유실물 특례)**:  
  - 게임 아이템이 해킹 또는 시스템 오류로 소멸된 경우, "금전"이 아닌 "데이터"로 보아 반환 청구가 어려울 수 있습니다.  
  - 단, 게임사가 오류를 인정한 경우, **약관에 따른 보상 절차**를 요구할 수 있습니다.  
- **전자상거래법 제17조(디지털콘텐츠 공급자의 의무)**:  
  - 게임사는 기술적 오류로 인한 피해를 최소화하기 위한 조치를 취해야 하며, 오류 발생 시 이용자 보호 조치를 마련해야 합니다.